In [1]:
%matplotlib qt
import sys
import mne
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.append(r"C:\Users\gautier\OneDrive - CentraleSupelec\3A - Master CNN\Supervised Project\pipeline project v0\scripts")
sys.path.append(r"C:\Users\gautier\OneDrive - CentraleSupelec\3A - Master CNN\Supervised Project\pipeline project v0\config")
import eeg_preprocessing as preprocessing
import eeg_decoding as decoding
import vst_config as config

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Channels Selection

On frontocentral channels

In [2]:
epochs_type = "start_prod"
baseline_duration = 0.1
epoch_duration = 4
pick_type = "frontocentral"

subject = "VST_02_X"

low_freq = 4

In [3]:
# Load epochs
conditions = ["LNS", "LHS", "SNS", "SHS", "LNL", "LHL", "SNL", "SHL"]
for condition in conditions:
    epochs = preprocessing.load_subject_epochs_by_type_and_condition(subject, condition, epochs_type, baseline_duration, epoch_duration, pick_type, verbose=False)
    covariance = mne.compute_covariance(epochs, method="auto", verbose=False)
    variances = np.diag(covariance.data)

    sorted_electrodes = np.argsort(variances)[::-1]

    print(variances[sorted_electrodes])
    print(f"Most to Least Variance for the {condition} condition", sorted_electrodes)

[6.05031496e-11 4.85189656e-11 3.79507208e-11 3.25708764e-11
 3.18872863e-11 3.18261462e-11 1.44106071e-11 1.11150685e-11
 1.01489727e-11]
Most to Least Variance for the LNS condition [7 4 1 0 3 8 2 5 6]
[4.10488256e-11 3.91078200e-11 2.98862979e-11 2.53064765e-11
 2.52835951e-11 2.47185391e-11 8.18133313e-12 7.86860454e-12
 6.89564021e-12]
Most to Least Variance for the LHS condition [7 4 1 0 8 3 2 5 6]
[4.92323293e-11 4.57542890e-11 3.95244850e-11 3.17030235e-11
 3.10616364e-11 2.85284498e-11 1.13031831e-11 1.04193366e-11
 8.06129240e-12]
Most to Least Variance for the SNS condition [7 4 1 0 8 3 2 5 6]
[4.10665546e-11 3.92360636e-11 3.13776506e-11 2.58071762e-11
 2.55794151e-11 2.37885375e-11 8.11992166e-12 7.92596689e-12
 6.65187740e-12]
Most to Least Variance for the SHS condition [7 4 1 0 8 3 2 5 6]
[4.85090117e-11 4.39758974e-11 3.74846595e-11 2.97259453e-11
 2.95795020e-11 2.79684676e-11 1.05210571e-11 9.75046778e-12
 8.20321619e-12]
Most to Least Variance for the LNL condition 

In [4]:
print(config.frontocentral_channels[7])

F2


On all channels

In [5]:
epochs_type = "start_prod"
baseline_duration = 0.1
epoch_duration = 4
pick_type = "all_channels"

subject = "VST_02_X"

low_freq = 4

In [6]:
# Load epochs
conditions = ["LNS", "LHS", "SNS", "SHS", "LNL", "LHL", "SNL", "SHL"]
for condition in conditions:
    epochs = preprocessing.load_subject_epochs_by_type_and_condition(subject, condition, epochs_type, baseline_duration, epoch_duration, pick_type, verbose=False)
    covariance = mne.compute_covariance(epochs, method="auto", verbose=False)
    variances = np.diag(covariance.data)

    sorted_electrodes = np.argsort(variances)[::-1]

    print(variances[sorted_electrodes])
    print(f"Most to Least Variance for the {condition} condition", sorted_electrodes)

[3.70587480e-09 6.39747321e-10 2.42099390e-10 2.25737786e-10
 1.87120066e-10 1.79509971e-10 1.43568520e-10 1.42266470e-10
 1.41912019e-10 1.31278708e-10 1.26852026e-10 1.24627940e-10
 1.22291956e-10 1.16786469e-10 1.12194486e-10 1.10881877e-10
 9.95093097e-11 9.82778379e-11 9.21712474e-11 9.13347446e-11
 9.13190272e-11 8.79352408e-11 8.78548215e-11 8.72964974e-11
 8.66413477e-11 8.49920176e-11 8.43998422e-11 7.99145565e-11
 7.34105737e-11 7.25946301e-11 7.23460395e-11 7.23156060e-11
 6.96850482e-11 6.10662760e-11 6.09603086e-11 6.08669504e-11
 6.06957683e-11 6.01426361e-11 5.98090146e-11 5.88414547e-11
 5.30093189e-11 5.04925702e-11 4.86964867e-11 4.86379013e-11
 4.78954575e-11 4.01542900e-11 3.80046792e-11 3.79980049e-11
 3.50880340e-11 3.25917576e-11 3.19039645e-11 3.18424485e-11
 2.94544827e-11 2.78450934e-11 2.49341477e-11 2.35689008e-11
 1.71230694e-11 1.67249149e-11 1.43198326e-11 1.10040318e-11
 1.00319962e-11 8.25918629e-12 6.88692208e-12]
Most to Least Variance for the LNS con

# Visualise PSD

## Utils

In [13]:
from mne.time_frequency import (
    tfr_array_morlet,
)

def get_spectrogram_from_epochs_array(epochs_array, low_freq, high_freq):
    frequencies = np.arange(low_freq, high_freq)  
    cycles_per_freq = 6
    
    power = tfr_array_morlet(epochs_array, sfreq=config.resample_sfreq, freqs=frequencies, n_cycles=cycles_per_freq, output='avg_power', n_jobs=-1)

    return power

In [14]:
def plot_psd_one_channel(psd):
    plt.figure(figsize=(8, 5))
    plt.plot(psd, label='PSD')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power Spectral Density')
    plt.title('Power Spectral Density')
    plt.legend()
    plt.grid(True)

    plt.show()

In [15]:
def plot_psd_all_channels(psd):

    average_psd = np.mean(psd, axis=0)
    psd_std = np.std(psd, axis=0)

    plt.figure(figsize=(8, 5))

    plt.plot(average_psd, label='Average PSD')

    plt.fill_between(
        np.arange(len(average_psd)), average_psd - psd_std, average_psd + psd_std, alpha=0.2, label='Standard Deviation'
    )

    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power Spectral Density (arbitrary units)')
    plt.title('Average PSD with Standard Deviation')
    plt.legend()
    plt.grid(True)

    plt.show()

## Removing 1/f trend

In [16]:
epochs_type = "start_prod"
baseline_duration = 0.1
epoch_duration = 4
pick_type = "frontocentral"

subject = "VST_02_X"
condition = "LNS"

low_freq = 4

In [17]:
condition_epochs = preprocessing.load_subject_epochs_by_type_and_condition(subject, condition, epochs_type, baseline_duration, epoch_duration, pick_type, verbose=False)
spectrogram = get_spectrogram_from_epochs_array(condition_epochs, low_freq, config.f_max_filtering)

psd = np.mean(spectrogram, axis=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    4.5s remaining:    5.7s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    4.5s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    4.9s finished


Plot the PSD of one channel

In [18]:
plot_psd_one_channel(psd[7])

Plot the PSD averaged on all channels

In [19]:
plot_psd_all_channels(psd)

Removing the 1/f trend

Using detrend in scipy

In [14]:
from scipy.signal import detrend

detrended_psd = detrend(psd, axis=1)
plot_psd_all_channels(detrended_psd)

Using the derivative

In [26]:
data = condition_epochs.get_data()
derived_data = np.diff(data, axis=2) / (1 / config.resample_sfreq)  
print(data.shape)
print(derived_data.shape)

derived_epochs = mne.EpochsArray(derived_data, epochs.info)

spectrogram = get_spectrogram_from_epochs_array(derived_epochs, low_freq, config.f_max_filtering)
psd = np.mean(spectrogram, axis=2)
plot_psd_all_channels(psd)

(104, 9, 1000)
(104, 9, 999)
Not setting metadata
104 matching events found
No baseline correction applied
0 projection items activated


C:\Users\gautier\AppData\Local\Temp/ipykernel_60628/3220476056.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = condition_epochs.get_data()
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.5s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.6s finished


# Visualise spectrograms

## Utils

In [15]:
from mne.baseline import rescale
from mne.viz import centers_to_edges

def plot_spectrogram(power, times, freqs, channel_name, channel_number, epochs_type, condition):

  # if no channel_name is given, we plot the average power over all channels
  if channel_name is None:
    selected_power = power.mean(axis=0)
    fig, ax = plt.subplots()
    x, y = centers_to_edges(times * 1000, freqs)
    mesh = ax.pcolormesh(x, y,selected_power, cmap="RdBu_r", vmin=np.min(selected_power), vmax=np.max(selected_power))
    ax.set_title(f"Averaged TFR on the {epochs_type} for the {condition} condition - Averaged on all channels")
    ax.set(ylim=freqs[[0, -1]], xlabel="Time (ms)")
    fig.colorbar(mesh)
  else:
    # rescale(power[channel_number], times, (0.0, 0.1), mode="mean", copy=False)
    selected_power = power[channel_number]
    fig, ax = plt.subplots()
    x, y = centers_to_edges(times * 1000, freqs)
    mesh = ax.pcolormesh(x, y,selected_power, cmap="RdBu_r", vmin=np.min(selected_power), vmax=np.max(selected_power))
    ax.set_title(f"Averaged TFR on the {epochs_type} for the {condition} condition - {channel_name} channel")
    ax.set(ylim=freqs[[0, -1]], xlabel="Time (ms)")
    fig.colorbar(mesh)

In [16]:
from mne.time_frequency import (
    tfr_array_morlet,
)

def get_spectrogram_from_epochs_array(epochs_array, low_freq, high_freq):
    frequencies = np.arange(low_freq, high_freq)  
    cycles_per_freq = 6
    
    power = tfr_array_morlet(epochs_array, sfreq=config.resample_sfreq, freqs=frequencies, n_cycles=cycles_per_freq, output='avg_power', n_jobs=-1)
    
    return power

def get_detrended_spectrogram_from_epochs_array(epochs_array, low_freq, high_freq):
    frequencies = np.arange(low_freq, high_freq)  
    cycles_per_freq = 6
    
    power = tfr_array_morlet(epochs_array, sfreq=config.resample_sfreq, freqs=frequencies, n_cycles=cycles_per_freq, output='avg_power', n_jobs=-1)
    # Detrend spectrogram on the frequency axis
    detrended_power = detrend(power, axis=1)
    return detrended_power

## Visualise spectrograms of one subject averaged on one condition

Spectrograms of single channel

In [17]:
epochs_type = "start_prod"
baseline_duration = 0.1
epoch_duration = 4
pick_type = "frontocentral"

subject = "VST_02_X"
condition = "LNS"
low_freq = 4

In [18]:
condition_epochs = preprocessing.load_subject_epochs_by_type_and_condition(subject, condition, epochs_type, baseline_duration, epoch_duration, pick_type, verbose=False)
spectrogram = get_spectrogram_from_epochs_array(condition_epochs, low_freq, config.f_max_filtering)

for channel_number, channel_name in enumerate(config.frontocentral_channels):
    times = np.arange(spectrogram.shape[2])/config.resample_sfreq
    frequencies = np.arange(low_freq, config.f_max_filtering)
    plot_spectrogram(spectrogram, times, frequencies, channel_name, channel_number, epochs_type, condition)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.4s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.4s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.6s finished


Channel 2 of frontocentral channels (FC1), seems to give good spectrograms. FCz does not

Detrended spectogram 

In [19]:
condition_epochs = preprocessing.load_subject_epochs_by_type_and_condition(subject, condition, epochs_type, baseline_duration, epoch_duration, pick_type, verbose=False)
detrended_spectrogram = get_detrended_spectrogram_from_epochs_array(condition_epochs, low_freq, config.f_max_filtering)
spectrogram = get_spectrogram_from_epochs_array(condition_epochs, low_freq, config.f_max_filtering)

# Visualising the detrended spectrogram on FC1 channel
times = np.arange(spectrogram.shape[2])/config.resample_sfreq
frequencies = np.arange(low_freq, config.f_max_filtering)
plot_spectrogram(spectrogram, times, frequencies, 'FC1', 1, epochs_type, condition)
plot_spectrogram(detrended_spectrogram, times, frequencies, 'FC1', 1, epochs_type, condition)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.4s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.4s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.3s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.4s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.5s finished


Detrended and normal spectrogram on all channels

In [20]:
# Visualising the detrended spectrogram on all channels of the frontocentral area
times = np.arange(spectrogram.shape[2])/config.resample_sfreq
frequencies = np.arange(low_freq, config.f_max_filtering)
plot_spectrogram(spectrogram, times, frequencies, None, None, epochs_type, condition)
plot_spectrogram(detrended_spectrogram, times, frequencies, None, None, epochs_type, condition)

## Compare spectrograms of 2 conditions

Plotting detrended and averaged spectrograms of a pair of conditions on FC1 channel

In [21]:
condition_1 = "LNS"
condition_2 = "LHS"
pick_type = "frontocentral"
subject = "VST_02_X"

epochs_type = "start_prod"
baseline_duration = 0.1
epoch_duration = 4

In [22]:
condition_1_epochs = preprocessing.load_subject_epochs_by_type_and_condition(subject, condition_1, epochs_type, baseline_duration, epoch_duration, pick_type, verbose=False)
spectrogram_condition_1 = get_spectrogram_from_epochs_array(condition_1_epochs, 5, config.f_max_filtering)

condition_2_epochs = preprocessing.load_subject_epochs_by_type_and_condition(subject, condition_2, epochs_type, baseline_duration, epoch_duration, pick_type, verbose=False)
spectrogram_condition_2 = get_detrended_spectrogram_from_epochs_array(condition_2_epochs, 5, config.f_max_filtering)

times = np.arange(spectrogram.shape[2])/config.resample_sfreq
frequencies = np.arange(5, config.f_max_filtering)
plot_spectrogram(spectrogram_condition_1, times, frequencies, None, None, epochs_type, condition_1)
plot_spectrogram(spectrogram_condition_2, times, frequencies, None, None, epochs_type, condition_2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.4s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.4s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.3s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.6s finished


# Power time courses on one channel

## Utils

In [23]:
def plot_time_course(power, times, power_range_name, channel_name, channel_number, epochs_type, condition):
   
  # TFR is computed from 4 to 40Hz
  if power_range_name == "theta":
    power_range = np.arange(4, 8, 1) - 4 
  elif power_range_name == "alpha":
    power_range = np.arange(8, 14, 1) - 4  
  elif power_range_name == "beta":
    power_range = np.arange(14, 30, 1) - 4  
  elif power_range_name == "gamma":
    power_range = np.arange(30, 40, 1) - 4 

  if channel_name is None:
    power_timecourse = np.mean(power[:, power_range, :], axis=(0, 1))
    fig, ax = plt.subplots(figsize=(12, 6))

    ax.plot(times, power_timecourse, label=condition, color='blue')
    ax.set_title(f"{power_range_name} time course on the {epochs_type} averaged on all channels for the {condition} condition")
    ax.set(ylabel=f"{power_range_name} power", xlabel="Time (ms)")
  else:
    power_timecourse = np.mean(power[channel_number][power_range, :], axis=0)
    fig, ax = plt.subplots(figsize=(12, 6))

    ax.plot(times, power_timecourse, label=condition, color='blue')
    ax.set_title(f"{power_range_name} time course on the {epochs_type} for channel {channel_name} for the {condition} condition")
    ax.set(ylabel=f"{power_range_name} power", xlabel="Time (ms)")

  ax.legend()

  plt.show()

## Time courses on two conditions

Computing theta time courses for several pairs of conditions on FC1 channel

In [24]:
epochs_type = "start_prod"
baseline_duration = 0.1
epoch_duration = 4
pick_type = "frontocentral"

subject = "VST_02_X"
condition_1 = "SNS"
condition_2 = "SNL"

low_freq = 4

In [25]:
condition_1_epochs = preprocessing.load_subject_epochs_by_type_and_condition(subject, condition_1, epochs_type, baseline_duration, epoch_duration, pick_type, verbose=False)
spectrogram_condition_1 = get_detrended_spectrogram_from_epochs_array(condition_1_epochs, 5, config.f_max_filtering)

condition_2_epochs = preprocessing.load_subject_epochs_by_type_and_condition(subject, condition_2, epochs_type, baseline_duration, epoch_duration, pick_type, verbose=False)
spectrogram_condition_2 = get_detrended_spectrogram_from_epochs_array(condition_2_epochs, 5, config.f_max_filtering)

times = np.arange(spectrogram.shape[2])/config.resample_sfreq

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.4s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.4s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.4s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.4s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.6s finished


In [26]:
# Power to analyze
power_range = "alpha"

Plot time courses for detretended spectrograms on single channel

In [27]:
channel_name = "F2"
channel_number = 7

In [28]:
plot_time_course(spectrogram_condition_1, times, power_range, channel_name, channel_number, epochs_type, condition_1)
plot_time_course(spectrogram_condition_2, times, power_range, channel_name, channel_number, epochs_type, condition_2)

Plot time courses for detretended spectrograms on all channels

In [29]:
plot_time_course(spectrogram_condition_1, times, power_range, None, None, epochs_type, condition_1)
plot_time_course(spectrogram_condition_2, times, power_range, None, None, epochs_type, condition_2)

C:\Users\gautier\AppData\Local\Temp/ipykernel_60528/1065190934.py:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(12, 6))


# Beta intercept of the four environments 

Computing the beta intercept at 0 second for the start_prod epoch for the four environments: LN, LH, SN and SH

## Utils

In [30]:
def get_power_intercept(power, power_range_name, time_intercept, channel_name, channel_number):
   
    # TFR is computed from 4 to 40Hz
    if power_range_name == "theta":
        power_range = np.arange(4, 8, 1) - 4 
    elif power_range_name == "alpha":
        power_range = np.arange(8, 14, 1) - 4  
    elif power_range_name == "beta":
        power_range = np.arange(14, 30, 1) - 4  
    elif power_range_name == "gamma":
        power_range = np.arange(30, 40, 1) - 4 

    power_timecourse = None
    if channel_name is None:
        power_timecourse = np.mean(power[:, power_range, :], axis=(0, 1))
    else:
        power_timecourse = np.mean(power[channel_number][power_range, :], axis=0)

    intercept = power_timecourse[time_intercept]
    return intercept   

## Plot intercepts

In [31]:
epochs_type = "start_prod"
baseline_duration = 0.1
epoch_duration = 4
pick_type = "frontocentral"
subject = "VST_02_X"

environments = ["LN", "LH", "SN", "SH"]
power_range = "beta"

low_freq = 4

In [32]:
# Retrieve the intercept for the four environments
intercepts = []


for environment in environments:
    environment_epochs_long = preprocessing.load_subject_epochs_by_type_and_condition(subject, environment+"L", epochs_type, baseline_duration, epoch_duration, pick_type, verbose=False)
    environment_epochs_short = preprocessing.load_subject_epochs_by_type_and_condition(subject, environment+"S", epochs_type, baseline_duration, epoch_duration, pick_type, verbose=False)
    environment_epochs = mne.concatenate_epochs([environment_epochs_long, environment_epochs_short], verbose=False)
    spectrogram = get_detrended_spectrogram_from_epochs_array(environment_epochs, low_freq, config.f_max_filtering)
    times = np.arange(spectrogram.shape[2])/config.resample_sfreq
    
    intercepts.append(get_power_intercept(spectrogram, power_range, 0,'F2', 7))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    1.0s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    1.1s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    1.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    1.0s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    1.2s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    1.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    1.2s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    1.3s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    2.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent worker

In [33]:
# Plot intercepts
fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(environments, intercepts)
ax.set_title(f"Intercept of the {power_range} power for the four environments")
ax.set(ylabel=f"{power_range} power", xlabel="Environment")
plt.show()

# Analysis on short/long produced times

The goal here is to associate each start_prod epoch with its relative produced duration compared to the goal. 
A produced duration will be considered short if it is n standard deviations away from the mean of the durations produced by the subject

In [2]:
epochs_type = "start_prod"
baseline_duration = 0.1
epoch_duration = 4
pick_type = "frontocentral"
subject = "VST_02_X"

environments = ["LN", "LH", "SN", "SH"]
conditions = ["LNS", "LHS", "SNS", "SHS", "LNL", "LHL", "SNL", "SHL"]

power_range = "beta"

low_freq = 4

## Analyse the produced durations distribution

Analyse distribution with epochs metadata

In [3]:
num_epochs = 0
for condition in conditions:
    epochs = preprocessing.load_subject_epochs_by_type_and_condition(subject, condition, epochs_type, baseline_duration, epoch_duration, pick_type, verbose=False)
    print("Number of epochs : "+str(epochs.__len__()) + " for condition " + condition)
    num_epochs += epochs.__len__()
print("\nTotal number of epochs: " + str(num_epochs))

Number of epochs : 104 for condition LNS
Number of epochs : 104 for condition LHS
Number of epochs : 104 for condition SNS
Number of epochs : 104 for condition SHS
Number of epochs : 104 for condition LNL
Number of epochs : 104 for condition LHL
Number of epochs : 104 for condition SNL
Number of epochs : 103 for condition SHL

Total number of epochs: 831


In [4]:
# Retrieve the produced duration distribution for all conditions
produced_durations_dict = {}
for condition in conditions:
    condition_epochs = preprocessing.load_subject_epochs_by_type_and_condition(subject, condition, epochs_type, baseline_duration, epoch_duration, pick_type, verbose=False)
    produced_durations_dict[condition] = condition_epochs.metadata["duration_produced"]

In [5]:
# Compute major statistics for each distribution
for condition in conditions:
    print(f"Condition {condition}:")
    print(f"Mean: {produced_durations_dict[condition].mean()}")
    print(f"Median: {produced_durations_dict[condition].median()}")
    print(f"Standard deviation: {produced_durations_dict[condition].std()}")
    print(f"Variance: {produced_durations_dict[condition].var()}")
    print(f"Skewness: {produced_durations_dict[condition].skew()}")
    print(f"Kurtosis: {produced_durations_dict[condition].kurtosis()}")
    print("\n")

Condition LNS:
Mean: 1.4974615384615384
Median: 1.46
Standard deviation: 0.26743342962884625
Variance: 0.07152063928304707
Skewness: 1.2385770441868147
Kurtosis: 2.708533317281989


Condition LHS:
Mean: 1.671153846153846
Median: 1.588
Standard deviation: 0.7016148193577051
Variance: 0.4922633547423451
Skewness: 3.7575196694132553
Kurtosis: 19.83102891627363


Condition SNS:
Mean: 1.3708846153846155
Median: 1.308
Standard deviation: 0.35031745125559977
Variance: 0.12272231665421951
Skewness: 0.266373222694754
Kurtosis: -0.6084547007875649


Condition SHS:
Mean: 1.6249615384615383
Median: 1.6
Standard deviation: 0.35652148788240245
Variance: 0.12710757132188202
Skewness: 0.26590494711641033
Kurtosis: 0.7678060424713076


Condition LNL:
Mean: 3.319576923076923
Median: 3.314
Standard deviation: 0.5884939259002665
Variance: 0.3463251008215084
Skewness: 0.0713900545659056
Kurtosis: -0.36211084467518706


Condition LHL:
Mean: 3.2854230769230766
Median: 3.212
Standard deviation: 0.590494521933

In [6]:
# Plot the produced duration for all conditions
fig, ax = plt.subplots(figsize=(8, 6))
for i, condition in enumerate(conditions):
    ax.boxplot(produced_durations_dict[condition], positions=[i], patch_artist=True)

ax.set_xticklabels(conditions)
ax.set_title("Produced Durations Box Plots")
plt.legend()
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Analyse distribution with behavioural metadata

In [7]:
import pandas as pd

metadata_2 = pd.read_csv(r"C:\Users\gautier\Downloads\clean_data_prod.csv")
metadata_2 = metadata_2[metadata_2["participant"] == "S02"]
print("Nombre total d'essais : ", metadata_2.shape[0])

metadata_2.loc[metadata_2["room_height"] == "Low", "room_height"] = "Normal"
metadata_2["condition_initials"] = metadata_2["room_size"].str[0].str.upper() + metadata_2["room_height"].str[0].str.upper() + metadata_2["dur_cond"].str[0].str.upper()

Nombre total d'essais :  824


In [12]:
# Retrieve the produced duration distribution for all conditions
produced_durations_dict_2 = {}
for condition in conditions:
    condition_df = metadata_2.loc[metadata_2["condition_initials"] == condition, "duration"]
    produced_durations_dict_2[condition] = condition_df
    print("Number of epochs : "+str(condition_df.shape[0]) + " for condition " + condition)

Number of epochs : 103 for condition LNS
Number of epochs : 104 for condition LHS
Number of epochs : 103 for condition SNS
Number of epochs : 103 for condition SHS
Number of epochs : 103 for condition LNL
Number of epochs : 102 for condition LHL
Number of epochs : 103 for condition SNL
Number of epochs : 103 for condition SHL


In [9]:
# Compute major statistics for each distribution
for condition in conditions:
    print(f"Condition {condition}:")
    print(f"Mean: {produced_durations_dict_2[condition].mean()}")
    print(f"Median: {produced_durations_dict_2[condition].median()}")
    print(f"Standard deviation: {produced_durations_dict_2[condition].std()}")
    print(f"Variance: {produced_durations_dict_2[condition].var()}")
    print(f"Skewness: {produced_durations_dict_2[condition].skew()}")
    print(f"Kurtosis: {produced_durations_dict_2[condition].kurtosis()}")
    print("\n")

Condition LNS:
Mean: 1.462459260194175
Median: 1.422363
Standard deviation: 0.2679702596813568
Variance: 0.07180806007369378
Skewness: 1.271197011195844
Kurtosis: 2.8096611224945134


Condition LHS:
Mean: 1.638482904807692
Median: 1.5555265
Standard deviation: 0.7016788430257083
Variance: 0.49235319874989647
Skewness: 3.755256549894285
Kurtosis: 19.8067863013939


Condition SNS:
Mean: 1.340301218446602
Median: 1.277832
Standard deviation: 0.35118827128939456
Variance: 0.12333320189123341
Skewness: 0.2498561954747845
Kurtosis: -0.6198711430555526


Condition SHS:
Mean: 1.5847096378640777
Median: 1.567383
Standard deviation: 0.34974777657374473
Variance: 0.12232350721827805
Skewness: 0.2501443113422723
Kurtosis: 0.8975613369882001


Condition LNL:
Mean: 3.2813827572815537
Median: 3.277832
Standard deviation: 0.5887090956712225
Variance: 0.3465783993260287
Skewness: 0.09290723135499794
Kurtosis: -0.3517978626365661


Condition LHL:
Mean: 3.256194529411765
Median: 3.1763915000000003
Standa

In [10]:
# Plot the produced duration for all conditions
fig, ax = plt.subplots(figsize=(8, 6))
for i, condition in enumerate(conditions):
    ax.boxplot(produced_durations_dict_2[condition], positions=[i], patch_artist=True)

ax.set_xticklabels(conditions)
ax.set_title("Produced Durations Box Plots")
plt.legend()
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Compare medians

In [11]:
for condition in conditions:
    print("Diff for " + condition + "  : " + str(produced_durations_dict[condition].median() - produced_durations_dict_2[condition].median()))

Diff for LNS  : 0.03763699999999992
Diff for LHS  : 0.032473500000000044
Diff for SNS  : 0.030167999999999973
Diff for SHS  : 0.03261700000000012
Diff for LNL  : 0.03616799999999998
Diff for LHL  : 0.03560849999999993
Diff for SNL  : 0.03554699999999977
Diff for SHL  : 0.05272100000000002


## Discriminate and analyse between short and long durations

The threshold is the number of standard deviations t

### Utils

In [ ]:
from mne.time_frequency import (
    tfr_array_morlet,
)

def get_full_spectrogram_from_epochs_array(epochs_array, low_freq, high_freq):
    frequencies = np.arange(low_freq, high_freq)  
    cycles_per_freq = 6
    
    power = tfr_array_morlet(epochs_array, sfreq=config.resample_sfreq, freqs=frequencies, n_cycles=cycles_per_freq, output='power', n_jobs=-1)
    
    return power

### Analysis